In [ ]:
# === SETUP ===
import carla
import numpy as np
import cv2
import random
import time
import threading
from pathlib import Path

Path("output").mkdir(parents=True, exist_ok=True)


In [ ]:
# === MAIN DATA COLLECTION FUNCTION ===
def main():
    # Connect to the CARLA Simulator
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)

    # Get world and blueprints
    world = client.get_world()
    blueprint_library = world.get_blueprint_library()
    
    # Choose red Tesla Model 3
    vehicle_bp = blueprint_library.find('vehicle.tesla.model3')
    vehicle_bp.set_attribute('color', '255,0,0')
    
    # Get a spawn point
    spawn_points = world.get_map().get_spawn_points()
    spawn_point = spawn_points[6]

    # Spawn vehicle
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    vehicle.set_autopilot(True)
    print(f"Spawned vehicle: {vehicle.type_id}")

    # Set up RGB camera
    camera_bp = blueprint_library.find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '320')
    camera_bp.set_attribute('image_size_y', '240')
    camera_bp.set_attribute('fov', '110')
    camera_bp.set_attribute('sensor_tick', '0.1')  # 10 FPS

    camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
    camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

    # Data buffers
    image_list = []
    angle_list = []

    # Callback for each frame
    def save_image(image):
        # Convert raw BGRA to numpy array
        array = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))
        gray = cv2.cvtColor(array, cv2.COLOR_BGRA2GRAY)
        resized = cv2.resize(gray, (160, 120))
        
        # Save image and current steering angle
        image_list.append(resized)
        angle_list.append(vehicle.get_control().steer)

        # Optional: Save preview image
        # cv2.imwrite(f"output/{image.frame:06d}.png", resized)

    # Start camera
    camera.listen(save_image)

    # Optional: Spectator follows vehicle
    spectator = world.get_spectator()
    def follow():
        while True:
            transform = vehicle.get_transform()
            spectator.set_transform(carla.Transform(
                transform.location + carla.Location(x=-6, z=3),
                transform.rotation))
            time.sleep(0.03)

    threading.Thread(target=follow, daemon=True).start()

    # Run for 30 seconds (should capture ~300 frames at 10 FPS)
    time.sleep(30)

    # Stop camera and save data
    camera.stop()
    np.save('output/images.npy', np.array(image_list))      # shape: (N, 120, 160)
    np.save('output/angles.npy', np.array(angle_list))      # shape: (N,)
    print(f"Saved {len(image_list)} images and angles to 'output/'")

    # Cleanup
    camera.destroy()
    vehicle.destroy()
    print("Vehicle destroyed")

In [ ]:
if __name__ == '__main__':
    main()